In [ ]:
import os, logging, warnings
warnings.filterwarnings("ignore", message="enable_nested_tensor is True, but self.use_nested_tensor is False")
warnings.filterwarnings("ignore", category=UserWarning, module='google.protobuf')
os.environ.update({
    "TF_CPP_MIN_LOG_LEVEL": "3",
    "TF_ENABLE_ONEDNN_OPTS": "0",
    "TOKENIZERS_PARALLELISM": "false",
    "CUDA_VISIBLE_DEVICES": "1"
})
logging.getLogger("tensorflow").setLevel(logging.FATAL)

os.environ["DEEPSEEK_API_KEY"] = r"sk-4a"

from typing import TypedDict,NotRequired
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.middleware import (
    wrap_tool_call, dynamic_prompt, ModelRequest, 
    AgentMiddleware,wrap_model_call, ModelResponse,wrap_tool_call
)
from langchain_deepseek import ChatDeepSeek
from langchain_core.tools import tool
from langchain.messages import ToolMessage,SystemMessage, HumanMessage
from FinTools import predict_stock_trend, evaluate_stock_risk, analyze_stock_valuation, comprehensive_stock_analysis,recommend_default_stocks
from FinRAG import fin_document_rag
from ReasearchRAG import research_document_rag

CUSTOM_TOOLS = [
    predict_stock_trend,
    evaluate_stock_risk,
    analyze_stock_valuation,
    recommend_default_stocks,
    comprehensive_stock_analysis,
    fin_document_rag,
    research_document_rag
]

@wrap_model_call
def dynamic_model_selection(request: ModelRequest, handler) -> ModelResponse:
    """Choose model based on conversation complexity."""
    message_count = len(request.state["messages"])
    if message_count > 10:
        model = llm
    else:
        model = llm
    return handler(request.override(model=model))
@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        print(f"{request.tool_call["id"]} met an error: {e}")
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    """Generate system prompt based on user role."""
    base_prompt = "You are a helpful assistant."
    if request.runtime.context:
        user_role = request.runtime.context.get("user_role", "user")
        if user_role == "expert":
            return f"{base_prompt} Provide detailed technical responses."
        elif user_role == "beginner":
            return f"{base_prompt} Explain concepts simply and avoid jargon."
    return base_prompt

class Context(TypedDict):
    user_role: NotRequired[str]
class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

# ===================== 3. 初始化LLM =====================
llm = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

#middleware顺序：befopre_agent,before_model,wrap_model_call,after_model, wrap_tool_call,before_model,wrap_model_call,after_model,after_agent
middleware = [
    dynamic_model_selection,
    user_role_prompt,
    handle_tool_errors
]

system_prompt=SystemMessage(
        content=[
            {
            "type": "text",
            "text": "You are an AI assistant tasked with analyzing problems.",
            }
        ]
    )

agent = create_agent(
    model=llm,
    tools=CUSTOM_TOOLS,
    middleware=middleware,
    context_schema=Context,
    system_prompt=system_prompt,
    # response_format=ToolStrategy(ContactInfo)
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "我是一名中年男性，请告诉我如何理财"}]},
    context={"user_role":"expert"}    
)
print("\n===== 最终回答 =====")
print(result)
print("第一个人处理完成")
result = agent.invoke(
    {"messages": [{"role": "user", "content": "我是一名在校大学生，比较胆小，请帮我分析一些比较好的股票"}]},
    context={"user_role":"expert"}    
)
print("\n===== 最终回答 =====")
print(result["messages"][-1].content)
print("第二个人处理完成")
result = agent.invoke(
    {"messages": [{"role": "user", "content": "我是一名在校大学生，比较激进，请帮我分析一些比较好的股票"}]},
    context={"user_role":"expert"}    
)
print("\n===== 最终回答 =====")
print(result["messages"][-1].content)
print("第三个人处理完成")

result = agent.invoke(
    {"messages": [
    {"role": "user", "content": "i want to know if recent 3 years, are there any research about the housing price in China ? please give me the entire examples you read, include title, abstract,year and journal name."}]},
    context={"user_role":"expert"}    
)
print("\n===== 最终回答 =====")
print(result["messages"][-1].content)

# print()

/home/wangyi/anaconda3/envs/langchain/lib/python3.12/site-packages/py_mini_racer/py_mini_racer.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

2025-12-26 18:57:52,919 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-12-26 18:57:59,949 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 18:58:26,193 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"



===== 最终回答 =====
{'messages': [HumanMessage(content='我是一名中年男性，请告诉我如何理财', additional_kwargs={}, response_metadata={}, id='79dcb727-bfc7-43bc-ab0d-6a4ae3fe4fd5'), AIMessage(content='您好！作为中年男性，理财规划确实需要特别关注。让我为您提供一些基础的投资原则和理财建议。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1084, 'total_tokens': 1165, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 1084}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '5f2f53d3-8698-402f-8cac-703618811450', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5a4d-0111-7c71-be1b-f0128762ac1b-0', tool_calls=[{'name': 'fin_document_rag', 'args': {'query': '基础投资原则、风险等级、市场分类、中年理财规划'}, 'id': 'call_00_J7icgcVPF030cHA2Hig4tNu5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1084, '

2025-12-26 18:58:31,072 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 18:58:37,981 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
/home/wangyi/anaconda3/envs/langchain/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/home/wangyi/anaconda3/envs/langchain/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
2025-12-26 18:58:42,388 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 18:58:46,013 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"


...

  0%|          | 0/57 [00:00<?, ?it/s]

2025-12-26 19:00:18,730 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:00:39,783 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"



===== 最终回答 =====
## 综合分析报告

### 1. 宁德时代（300750.SZ） - 新能源电池龙头
**适合激进投资者的特点：**
- **高波动性**：年化波动率32.73%，高风险等级
- **成长性**：新能源行业龙头，长期成长空间大
- **短期趋势**：预测下跌0.31%，置信度中等
- **估值**：PE 26.14，PB 5.44（估值偏高）
- **风险**：历史最大回撤-66.83%，单日最大潜在损失-4.82%

### 2. 恒生电子（600570.SH） - 金融科技
**适合激进投资者的特点：**
- **更高波动性**：年化波动率34.06%，风险更高
- **科技属性**：金融科技概念，成长性强
- **短期趋势**：预测上涨0.68%，但置信度低
- **估值**：PE 93.57，PB 6.28（估值很高）
- **风险**：历史最大回撤-75.99%，单日最大潜在损失-5.01%

### 3. 英伟达（NVDA） - AI芯片龙头
（由于数据源限制，无法获取详细分析，但作为激进投资者的典型代表）
- **极高成长性**：AI芯片全球龙头
- **高波动性**：科技股典型特征
- **长期前景**：AI浪潮核心受益者

## 投资建议（针对激进型大学生投资者）

### 风险控制策略：
1. **仓位管理**：单只股票仓位不超过总资金的10-15%
2. **止损设置**：设置5-8%的止损线
3. **分批建仓**：不要一次性投入全部资金
4. **分散投资**：至少选择2-3只不同行业的股票

### 具体操作建议：
1. **宁德时代**：等待回调机会，PB偏高需谨慎，可在PB<4时考虑分批建仓
2. **恒生电子**：估值过高，建议观望，等待估值回归合理区间
3. **英伟达**：可考虑通过港股通或美股账户配置，但需注意汇率风险

### 适合激进投资者的其他方向：
- **科创板/创业板**：高成长性科技股
- **新能源/半导体**：政策支持，长期成长空间大
- **生物医药**：创新药、医疗器械等高成长领域

### 重要提醒：
1. **学生投资原则**：用闲钱投资，不影响学业和生活
2. **学习为主**：投资是学习过程，不要追求短期暴利
3. **长期视角**：激进不等于频繁交易，要有长

2025-12-26 19:00:43,092 - INFO - Load pretrained SentenceTransformer: /mnt/data/models/all-roberta-large-v1
2025-12-26 19:00:44,115 - INFO - Loading all indices.
2025-12-26 19:00:44,530 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:00:52,777 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:00:56,218 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:01:06,556 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:01:10,213 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:01:13,646 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:01:16,873 - INFO - HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
2025-12-26 19:01:24,829 - INFO - HTTP Request


===== 最终回答 =====
Based on my searches through the research documents, I can provide you with the following research papers about housing prices in China from recent years:

## Research Papers on Housing Prices in China (2020-2023)

### 1. **Economic Slowdown and Housing Dynamics in China: A Tale of Two Investments by Firms**
- **Journal:** Journal of Money, Credit and Banking
- **Year:** 2022
- **Abstract:** The study examines China's housing boom and economic slowdown using a dynamic New Keynesian model. It introduces a channel where firms dynamically choose between physical and housing investment. Housing assets provide a positive return and serve as collateral for external finance. The model finds that a negative productivity shock lowers the relative return of production capital, increasing firm demand for housing and causing a boom. Rising house prices then create competing effects on real investment: a collateral effect that increases firm leverage and a crowding-out effect that

In [2]:
print(result)


{'messages': [HumanMessage(content='i want to know if recent 3 years, are there any research about the housing price in China ? please give me the entire examples you read, include title, abstract,year and journal name.', additional_kwargs={}, response_metadata={}, id='b274c46f-1a0e-4700-83d5-3c0b7fca1c4e'), AIMessage(content="I'll help you find research about housing prices in China from the last 3 years. Let me search the research documents for you.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 85, 'prompt_tokens': 1115, 'total_tokens': 1200, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1024}, 'prompt_cache_hit_tokens': 1024, 'prompt_cache_miss_tokens': 91}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '4b8e0e5e-d852-42d2-8b39-b8650debea32', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run

{'messages': 
[
HumanMessage(content='i want to know if recent 3 years, are there any research about the housing price in China ? please give me the entire examples you read, include title, abstract,year and journal name.', additional_kwargs={}, response_metadata={}, id='5c95be2b-600f-4bc6-bd28-1cfe23ea90c2'), 

AIMessage(content="I'll help you find research about housing prices in China from the last 3 years. Let me search for relevant academic documents.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 1110, 'total_tokens': 1201, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1088}, 'prompt_cache_hit_tokens': 1088, 'prompt_cache_miss_tokens': 22}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'ac01ad66-a472-4e94-931e-b74e1d0cfe86', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5063-36f8-7881-9f8d-94bebc206fec-0', tool_calls=[{'name': 'research_document_rag', 'args': {'query': 'housing price in China research papers from last 3 years, include title, abstract, year, journal name'}, 'id': 'call_00_GA4WM8V47BlnROrk93d7ga9h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1110, 'output_tokens': 91, 'total_tokens': 1201, 'input_token_details': {'cache_read': 1088}, 'output_token_details': {}}), 

ToolMessage(content="Based on the provided information, one research paper from the last three years addresses housing prices in China.\n\n**Title:** Economic Slowdown and Housing Dynamics in China: A Tale of Two Investments by Firms  \n**Year:** 2022  \n**Journal Name:** Journal of Money, Credit and Banking  \n**Abstract:** This study examines China's housing boom and economic slowdown using a dynamic New Keynesian model. It introduces a channel where firms dynamically choose between physical and housing investment. Housing assets provide a positive return and can serve as collateral. The model shows that a negative productivity shock lowers the relative return of production capital, increasing firm demand for housing and driving a boom in house prices. Rising house prices then create competing effects on real investment: a collateral effect that increases firm leverage and a crowding-out effect that reduces demand for physical capital. The quantitative analysis for China indicates the crowding-out effect dominates, leading to countercyclical housing prices. The paper concludes that capital subsidization policies targeting house prices are more effective than other macroeconomic policies.", name='research_document_rag', id='a4526e53-653e-4297-953b-6ad73ef3af0e', tool_call_id='call_00_GA4WM8V47BlnROrk93d7ga9h'), 

AIMessage(content='Let me search for more comprehensive information about housing price research in China from the last 3 years.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 1421, 'total_tokens': 1505, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1152}, 'prompt_cache_hit_tokens': 1152, 'prompt_cache_miss_tokens': 269}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'fcccef0b-b9df-4bd4-b259-4f87422b6703', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5063-676d-7223-8f74-77b76c1dd526-0', tool_calls=[{'name': 'research_document_rag', 'args': {'query': 'China housing prices research 2021 2022 2023 2024 academic papers studies analysis'}, 'id': 'call_00_TJmjeyhQ6XnFa7QXIq0K7ufQ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1421, 'output_tokens': 84, 'total_tokens': 1505, 'input_token_details': {'cache_read': 1152}, 'output_token_details': {}}), 

ToolMessage(content='The provided context includes a 2024 study examining foreign Chinese housing purchases in the U.S. market after 2008, noting a surge in capital inflows that concentrate in neighborhoods with existing ethnic Chinese populations. This research finds these inflows stimulate local employment through a housing net worth channel but also lead to gentrification by displacing lower-income residents. Additionally, a 2021 literature survey reviews recent studies connecting housing markets with credit, expectations, and financial stability, highlighting international diversity in house price behavior and policy impacts. No specific academic papers analyzing housing prices within China for the years 2021-2024 are detailed in the context.', name='research_document_rag', id='eb4ef1bb-dbfd-44fc-b807-3465c5758fec', tool_call_id='call_00_TJmjeyhQ6XnFa7QXIq0K7ufQ'), 

AIMessage(content='Let me search more specifically for research papers about Chinese housing prices in recent years.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 1646, 'total_tokens': 1732, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1472}, 'prompt_cache_hit_tokens': 1472, 'prompt_cache_miss_tokens': 174}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'decbefb1-7c61-4a80-96f7-d8a3e6e14f7c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5063-8bf1-7120-b700-a3ce04f8e85e-0', tool_calls=[{'name': 'research_document_rag', 'args': {'query': '"Chinese housing prices" "China real estate" "property prices China" research papers 2021 2022 2023'}, 'id': 'call_00_3WbWDiUIyIJWEsWombqKXFQZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1646, 'output_tokens': 86, 'total_tokens': 1732, 'input_token_details': {'cache_read': 1472}, 'output_token_details': {}}), 

ToolMessage(content="The provided context does not contain specific research papers from 2021, 2022, or 2023 that directly analyze Chinese housing prices, China's real estate market, or property prices within China. The information focuses on other topics, such as the impact of foreign Chinese capital on U.S. housing markets, a broad survey of housing price literature, and studies in financial regulation, healthcare, and historical economics.", name='research_document_rag', id='551a144b-b284-42e1-a0bb-c21413612fc8', tool_call_id='call_00_3WbWDiUIyIJWEsWombqKXFQZ'), 

AIMessage(content='Let me try a broader search to find any research related to housing or real estate in China from recent years.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 1830, 'total_tokens': 1911, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1728}, 'prompt_cache_hit_tokens': 1728, 'prompt_cache_miss_tokens': 102}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'e3a64fcf-661a-484e-903a-36832d3d2930', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b5063-aaea-7831-b82d-ab6566833fb8-0', tool_calls=[{'name': 'research_document_rag', 'args': {'query': 'China real estate market housing property research academic papers 2020-2024'}, 'id': 'call_00_MPy8dqfkRQSsOkUlragMW6YZ', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1830, 'output_tokens': 81, 'total_tokens': 1911, 'input_token_details': {'cache_read': 1728}, 'output_token_details': {}}), 

ToolMessage(content='Based on the provided information, there is one relevant academic paper from 2024 that directly examines the Chinese real estate market. The study analyzes transaction-level data to document a surge in foreign Chinese housing purchases in the U.S. after 2008, noting a concentration in neighborhoods with historical ethnic Chinese populations. This capital inflow is found to increase local employment through a housing net worth channel but also leads to the displacement of lower-income residents, illustrating effects of gentrification.', name='research_document_rag', id='a6947cf1-0fa9-48e5-b2d6-77eaab308f62', tool_call_id='call_00_MPy8dqfkRQSsOkUlragMW6YZ'), 

AIMessage(content="Based on my searches through the research document database, I can provide you with the following information about recent research on housing prices in China:\n\n## Research Papers on Housing Prices in China (Last 3 Years)\n\n### 1. **Primary Research Paper Found:**\n\n**Title:** Economic Slowdown and Housing Dynamics in China: A Tale of Two Investments by Firms  \n**Year:** 2022  \n**Journal Name:** Journal of Money, Credit and Banking  \n**Abstract:** This study examines China's housing boom and economic slowdown using a dynamic New Keynesian model. It introduces a channel where firms dynamically choose between physical and housing investment. Housing assets provide a positive return and can serve as collateral. The model shows that a negative productivity shock lowers the relative return of production capital, increasing firm demand for housing and driving a boom in house prices. Rising house prices then create competing effects on real investment: a collateral effect that increases firm leverage and a crowding-out effect that reduces demand for physical capital. The quantitative analysis for China indicates the crowding-out effect dominates, leading to countercyclical housing prices. The paper concludes that capital subsidization policies targeting house prices are more effective than other macroeconomic policies.\n\n### 2. **Related Research (2024):**\n\n**Title:** [Not specified in detail, but related to foreign Chinese housing purchases]\n**Year:** 2024  \n**Focus:** Analyzes foreign Chinese housing purchases in the U.S. market after 2008, noting a surge in capital inflows that concentrate in neighborhoods with existing ethnic Chinese populations. This research finds these inflows stimulate local employment through a housing net worth channel but also lead to gentrification by displacing lower-income residents.\n\n### 3. **Literature Survey (2021):**\n\n**Year:** 2021  \n**Type:** Literature survey reviewing recent studies connecting housing markets with credit, expectations, and financial stability, highlighting international diversity in house price behavior and policy impacts.\n\n## Limitations and Observations:\n\n1. **Limited Direct Research:** The database search revealed relatively few recent (2021-2024) academic papers specifically analyzing Chinese housing prices within China.\n\n2. **Research Focus Shift:** Much of the recent research appears to focus on:\n   - The impact of Chinese capital on foreign housing markets (particularly U.S.)\n   - Theoretical modeling of housing market dynamics\n   - Broader literature surveys rather than empirical studies of Chinese housing prices\n\n3. **Possible Reasons for Limited Results:**\n   - The research document database may have limited coverage of recent Chinese housing market studies\n   - Many studies on Chinese housing prices might be published in Chinese-language journals not included in the database\n   - Research may be more focused on policy analysis, market reports, or working papers rather than peer-reviewed journal articles\n\n## Recommendations for Further Research:\n\nIf you need more comprehensive information about recent research on Chinese housing prices, you might want to:\n1. Search specialized databases like CNKI (China National Knowledge Infrastructure) for Chinese-language research\n2. Look for working papers from institutions like the People's Bank of China, Chinese Academy of Social Sciences, or major Chinese universities\n3. Check policy research from organizations like the International Monetary Fund (IMF) or World Bank that often analyze Chinese housing markets\n4. Search for conference papers from real estate and economics conferences\n\nThe 2022 paper in the Journal of Money, Credit and Banking appears to be the most directly relevant peer-reviewed research on Chinese housing prices from the last 3 years in the available database.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 697, 'prompt_tokens': 2020, 'total_tokens': 2717, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 1856}, 'prompt_cache_hit_tokens': 1856, 'prompt_cache_miss_tokens': 164}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': '26271c04-ff20-46aa-bf76-74884731fc1e', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b5063-cc80-7fd3-8d1f-dad7cfb0a186-0', usage_metadata={'input_tokens': 2020, 'output_tokens': 697, 'total_tokens': 2717, 'input_token_details': {'cache_read': 1856}, 'output_token_details': {}})]}
